## Imports

In [92]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [93]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 5th Nov. 12th


In [94]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [95]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [96]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [97]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [98]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-4-2024,Condo/Co-op,3421 Spanish 324 Trl #324,Delray Beach,FL,33483.0,430000.0,2.0,2.0,Tropic Harbor Condo Part Four,1000.0,NaN,1969.0,NaN,430.0,824.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/3421-Sp...,Beaches MLS,RX-11034000,N,Y,26.422968,-80.069513


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [99]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [100]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
65,PAST SALE,2024-11-08,Condo/Co-op,448 Fanshaw K,Boca Raton,FL,33434.0,85000.0,2.0,1.5,Century Village,835.0,NaN,1980.0,NaN,102.0,431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/448-Fansh...,Beaches MLS,RX-10943508,N,Y,26.384826,-80.174459
16,PAST SALE,2024-11-08,Condo/Co-op,2000 S Ocean Blvd Unit 9-D,Boca Raton,FL,33432.0,1225000.0,2.0,2.0,Whitehall Condo,1236.0,NaN,1971.0,NaN,991.0,1478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024359,N,Y,26.328302,-80.074423
34,PAST SALE,2024-11-08,Condo/Co-op,230 Horizons E #107,Boynton Beach,FL,33435.0,140000.0,1.0,1.0,Sterling Village 230 Cond Decl IN,570.0,NaN,1968.0,NaN,246.0,551.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/230-Ho...,Beaches MLS,RX-11012824,N,Y,26.525150,-80.054830
40,PAST SALE,2024-11-08,Condo/Co-op,5650 Camino Del Sol #203,Boca Raton,FL,33433.0,345000.0,2.0,2.0,Camino Real Village Condo,1254.0,871934.0,1981.0,NaN,275.0,683.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/5650-Cami...,Beaches MLS,RX-11007660,N,Y,26.341918,-80.132155
31,PAST SALE,2024-11-08,Condo/Co-op,6699 NW 2nd Ave #312,Boca Raton,FL,33487.0,226800.0,1.0,2.0,Boca Teeca Condo 6,1050.0,NaN,1976.0,NaN,216.0,393.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/6699-NW-2...,Beaches MLS,RX-11015227,N,Y,26.408885,-80.083153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,PAST SALE,2024-11-04,Condo/Co-op,3630 Gardens Pkwy Unit 102c,Palm Beach Gardens,FL,33410.0,645000.0,2.0,2.5,Landmark AT The Gardens,1878.0,NaN,2007.0,NaN,343.0,2124.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/3...,Beaches MLS,RX-11014167,N,Y,26.849669,-80.090815
15,PAST SALE,2024-11-04,Condo/Co-op,5656 Via Delray Unit C,Delray Beach,FL,33484.0,273000.0,2.0,2.0,Palm Greens AT Villa Del Ray Condo II,1250.0,NaN,1974.0,NaN,218.0,530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/5656-Vi...,Beaches MLS,RX-11026944,N,Y,26.476103,-80.132374
20,PAST SALE,2024-11-04,Condo/Co-op,3186 Via Poinciana #202,Lake Worth,FL,33467.0,155000.0,1.0,1.5,Poinciana Lakes Condo,834.0,NaN,1984.0,NaN,186.0,312.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/3186-Via-...,Beaches MLS,RX-11013130,N,Y,26.629963,-80.155094
45,PAST SALE,2024-11-04,Condo/Co-op,106 Brackenwood Rd,Palm Beach Gardens,FL,33418.0,370000.0,2.0,2.0,Golf Villas at Pga National,1366.0,NaN,1981.0,NaN,271.0,667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-10999052,N,Y,26.831623,-80.148800


In [101]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [102]:
df_top_ten = df.head(10)

## Collect Agent Information

In [103]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [104]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [105]:
response_df = pd.DataFrame(response_list)

In [106]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [107]:
df_top_ten = merged_df

## Current Week's Values

In [108]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

65


In [109]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$427,088


In [110]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$297


In [111]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$27,760,700


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [113]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
114
Input Previous Week Condo Average Sales Price:
440082
Input Previous Week Condo Average PSF:
294
Input Previous Week Condo Sales total (ex: 198_000_000)
50_200_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [114]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [115]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [116]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [117]:
pd.set_option('display.max_columns',None)

In [118]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-05,Condo/Co-op,350 S Ocean Blvd Unit 6-A,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10995846,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,Scott Eckert,RPE Realty,None,None,1,orange


In [119]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [120]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-05,Condo/Co-op,350 S Ocean Blvd Unit 6-A,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10995846,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,Scott Eckert,RPE Realty,None,None,1,orange
1,PAST SALE,2024-11-06,Condo/Co-op,2150 S Ocean Blvd Unit 2c,Delray Beach,FL,33483.0,1725000.0,3.0,3.5,Bermuda High West One And Two Condo,2352.0,NaN,1979.0,NaN,733.0,1983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Beaches MLS,RX-11005988,N,Y,26.434010,-80.063828,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Judith Troup,Regency Realty Services,None,None,2,blue
2,PAST SALE,2024-11-05,Condo/Co-op,1000 Lowry St Ph -C,Delray Beach,FL,33483.0,1450000.0,2.0,2.5,Delray Summit Condo,2110.0,NaN,1966.0,NaN,687.0,2290.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Beaches MLS,RX-11028066,N,Y,26.463100,-80.062729,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Sally Areson,Dunhill 100 LLC,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,3,blue
3,PAST SALE,2024-11-06,Condo/Co-op,200 Macfarlane Dr #705,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Seagate Towers,1760.0,NaN,1970.0,NaN,739.0,1522.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,RX-10961305,N,Y,26.457685,-80.063664,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Erin Omiatek,One Sotheby's International Realty,None,None,Mickel Averick,The Corcoran Group,None,None,4,blue
4,PAST SALE,2024-11-08,Condo/Co-op,2000 S Ocean Blvd Unit 9-D,Boca Raton,FL,33432.0,1225000.0,2.0,2.0,Whitehall Condo,1236.0,NaN,1971.0,NaN,991.0,1478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024359,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Stuart Rosenberg,The Keyes Company (PBG),Maude Eaton,The Keyes Company (PBG),Torm Martinson,Campbell&Rosemurgy Real Estate,None,None,5,blue
5,PAST SALE,2024-11-05,Condo/Co-op,3210 S Ocean Blvd #604,Highland Beach,FL,33487.0,1065000.0,3.0,3.0,Villa Costa Condo,2108.0,NaN,1991.0,NaN,505.0,2707.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3210-...,Beaches MLS,RX-10982364,N,Y,26.417636,-80.065297,https://www.redfin.com/FL/Highland-Beach/3210-...,Margaret Empsall,John P. O'Grady LLC,Sandra O'Neill-Shine,John P. O'Grady LLC,Amy Blake,John P. O'Grady LLC,Kathya Seckinger,John P. O'Grady LLC,6,blue
6,PAST SALE,2024-11-06,Condo/Co-op,550 Okeechobee Blvd #910,West Palm Beach,FL,33401.0,950000.0,2.0,2.0,Cityplace South Tower Condo,1694.0,NaN,2008.0,NaN,561.0,1670.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-11019913,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Thalia Arellano,Sutter & Nugent LLC,None,None,Robert Anderson,"Tortoise Realty Group, LLC",None,None,7,blue
7,PAST SALE,2024-11-05,Condo/Co-op,1200 S Flagler Dr #1706,West Palm Beach,FL,33401.0,930000.0,2.0,2.0,Trianon Condo,1247.0,NaN,1972.0,NaN,746.0,1880.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1200...,Beaches MLS,F10441726,N,Y,26.702722,-80.049673,https://www.redfin.com/FL/West-Palm-Beach/1200...,Chris Cusimano,Keller Williams Realty Boca Raton,Marisa Hold Steller,Keller Williams Realty Boca Raton,None,Brown Harris Stevens of PB,None,None,8,blue
8,PAST SAL

In [122]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'Nov. 3rd — Nov. 11th')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [123]:
m.save('index.html')

## Data snagger

In [124]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [125]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-05,Condo/Co-op,350 S Ocean Blvd Unit 6-A,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10995846,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,Scott Eckert,RPE Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [126]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_11112024


# CREATE TEMPLATE 

In [127]:
muni_set = set(df_top_ten['CITY'])

In [128]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [129]:
df_top_ten.reset_index(inplace=True,drop=True)

In [130]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [131]:
top_sale

'Beresford at 350 S Ocean Blvd Unit 6-A in Boca Raton'

In [132]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [133]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [134]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-05,Condo/Co-op,350 S Ocean Blvd Unit 6-A,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10995846,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,Scott Eckert,RPE Realty,None,None,1,orange,Beresford at 350 S Ocean Blvd Unit 6-A in Boca...
1,PAST SALE,2024-11-06,Condo/Co-op,2150 S Ocean Blvd Unit 2c,Delray Beach,FL,33483.0,1725000.0,3.0,3.5,Bermuda High West One And Two Condo,2352.0,NaN,1979.0,NaN,733.0,1983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Beaches MLS,RX-11005988,N,Y,26.434010,-80.063828,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Judith Troup,Regency Realty Services,None,None,2,blue,Bermuda High West One And Two Condo at 2150 S ...
2,PAST SALE,2024-11-05,Condo/Co-op,1000 Lowry St Ph -C,Delray Beach,FL,33483.0,1450000.0,2.0,2.5,Delray Summit Condo,2110.0,NaN,1966.0,NaN,687.0,2290.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Beaches MLS,RX-11028066,N,Y,26.463100,-80.062729,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Sally Areson,Dunhill 100 LLC,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,3,blue,Delray Summit Condo at 1000 Lowry St Ph -C in ...
3,PAST SALE,2024-11-06,Condo/Co-op,200 Macfarlane Dr #705,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Seagate Towers,1760.0,NaN,1970.0,NaN,739.0,1522.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,RX-10961305,N,Y,26.457685,-80.063664,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Erin Omiatek,One Sotheby's International Realty,None,None,Mickel Averick,The Corcoran Group,None,None,4,blue,Seagate Towers at 200 Macfarlane Dr #705 in De...
4,PAST SALE,2024-11-08,Condo/Co-op,2000 S Ocean Blvd Unit 9-D,Boca Raton,FL,33432.0,1225000.0,2.0,2.0,Whitehall Condo,1236.0,NaN,1971.0,NaN,991.0,1478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024359,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Stuart Rosenberg,The Keyes Company (PBG),Maude Eaton,The Keyes Company (PBG),Torm Martinson,Campbell&Rosemurgy Real Estate,None,None,5,blue,Whitehall Condo at 2000 S Ocean Blvd Unit 9-D ...
5,PAST SALE,2024-11-05,Condo/Co-op,3210 S Ocean Blvd #604,Highland Beach,FL,33487.0,1065000.0,3.0,3.0,Villa Costa Condo,2108.0,NaN,1991.0,NaN,505.0,2707.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3210-...,Beaches MLS,RX-10982364,N,Y,26.417636,-80.065297,https://www.redfin.com/FL/Highland-Beach/3210-...,Margaret Empsall,John P. O'Grady LLC,Sandra O'Neill-Shine,John P. O'Grady LLC,Amy Blake,John P. O'Grady LLC,Kathya Seckinger,John P. O'Grady LLC,6,blue,Villa Costa Condo at 3210 S Ocean Blvd #604 in...
6,PAST SALE,2024-11-06,Condo/Co-op,550 Okeechobee Blvd #910,West Palm Beach,FL,33401.0,950000.0,2.0,2.0,Cityplace South Tower Condo,1694.0,NaN,2008.0,NaN,561.0,1670.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-11019913,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Thalia Arellano,Sutter & Nugent LLC,None,None,Robert Anderson,"Tortoise Realty Group, LLC",None,None,7,blue,Cityplace South Tower Condo at 550 Okeechobee ...
7,PAST SALE,2024-11-05,Condo/Co-op,1200 S Flagler Dr #1706,West Palm Beach,FL,33401.0,930000.0,2.0,2.0,Trianon Con

In [135]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Beresford closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $750,000M to $3,650,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Highland Beach, Delray Beach, Boca Raton, Palm Beach, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 65 condo sales totaling $27,760,700 million from Nov. 5th to Nov. 12th. The previous week, brokers closed 114 condo sales totaling $50,200,000.

Last week’s units sold for an average of $427,088, lower than the $440,082 average price f

In [136]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [137]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [138]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [139]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-05,Condo/Co-op,350 S Ocean Blvd Unit 6-A,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10995846,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,Scott Eckert,RPE Realty,None,None,1,orange,Beresford at 350 S Ocean Blvd Unit 6-A in Boca...
1,PAST SALE,2024-11-06,Condo/Co-op,2150 S Ocean Blvd Unit 2c,Delray Beach,FL,33483.0,1725000.0,3.0,3.5,Bermuda High West One And Two Condo,2352.0,NaN,1979.0,NaN,733.0,1983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Beaches MLS,RX-11005988,N,Y,26.434010,-80.063828,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Judith Troup,Regency Realty Services,None,None,2,blue,Bermuda High West One And Two Condo at 2150 S ...
2,PAST SALE,2024-11-05,Condo/Co-op,1000 Lowry St Ph -C,Delray Beach,FL,33483.0,1450000.0,2.0,2.5,Delray Summit Condo,2110.0,NaN,1966.0,NaN,687.0,2290.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Beaches MLS,RX-11028066,N,Y,26.463100,-80.062729,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Sally Areson,Dunhill 100 LLC,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,3,blue,Delray Summit Condo at 1000 Lowry St Ph -C in ...
3,PAST SALE,2024-11-06,Condo/Co-op,200 Macfarlane Dr #705,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Seagate Towers,1760.0,NaN,1970.0,NaN,739.0,1522.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,RX-10961305,N,Y,26.457685,-80.063664,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Erin Omiatek,One Sotheby's International Realty,None,None,Mickel Averick,The Corcoran Group,None,None,4,blue,Seagate Towers at 200 Macfarlane Dr #705 in De...
4,PAST SALE,2024-11-08,Condo/Co-op,2000 S Ocean Blvd Unit 9-D,Boca Raton,FL,33432.0,1225000.0,2.0,2.0,Whitehall Condo,1236.0,NaN,1971.0,NaN,991.0,1478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024359,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Stuart Rosenberg,The Keyes Company (PBG),Maude Eaton,The Keyes Company (PBG),Torm Martinson,Campbell&Rosemurgy Real Estate,None,None,5,blue,Whitehall Condo at 2000 S Ocean Blvd Unit 9-D ...
5,PAST SALE,2024-11-05,Condo/Co-op,3210 S Ocean Blvd #604,Highland Beach,FL,33487.0,1065000.0,3.0,3.0,Villa Costa Condo,2108.0,NaN,1991.0,NaN,505.0,2707.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3210-...,Beaches MLS,RX-10982364,N,Y,26.417636,-80.065297,https://www.redfin.com/FL/Highland-Beach/3210-...,Margaret Empsall,John P. O'Grady LLC,Sandra O'Neill-Shine,John P. O'Grady LLC,Amy Blake,John P. O'Grady LLC,Kathya Seckinger,John P. O'Grady LLC,6,blue,Villa Costa Condo at 3210 S Ocean Blvd #604 in...
6,PAST SALE,2024-11-06,Condo/Co-op,550 Okeechobee Blvd #910,West Palm Beach,FL,33401.0,950000.0,2.0,2.0,Cityplace South Tower Condo,1694.0,NaN,2008.0,NaN,561.0,1670.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-11019913,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Thalia Arellano,Sutter & Nugent LLC,None,None,Robert Anderson,"Tortoise Realty Group, LLC",None,None,7,blue,Cityplace South Tower Condo at 550 Okeechobee ...
7,PAST SALE,2024-11-05,Condo/Co-op,1200 S Flagler Dr #1706,West Palm Beach,FL,33401.0,930000.0,2.0,2.0,Trianon Con

In [140]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Beresford closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $750,000M to $3,650,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Highland Beach, Delray Beach, Boca Raton, Palm Beach, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 65 condo sales totaling $27,760,700 million from Nov. 5th to Nov. 12th. The previous week, brokers closed 114 condo sales totaling $50,200,000.

Last week’s units sold for an average of $427,088, lower than the $440,082 average price f

In [166]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Boca-Raton/350-S-Ocean-Blvd-33432/unit-6-A/home/178993151


In [167]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Delray-Beach/2150-S-Ocean-Blvd-33483/unit-2C/home/42370940


In [168]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach/2165-Ibis-Isle-Rd-33480/unit-3/home/42438280


In [169]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Highland-Beach/3908-S-Ocean-Blvd-33487/unit-564/home/42397161


In [170]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/350-S-Ocean-Blvd-33432/unit-6-A/home/178993151


In [171]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Highland-Beach/3908-S-Ocean-Blvd-33487/unit-564/home/42397161


In [172]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-05,Condo/Co-op,350 S Ocean Blvd Unit 6-A,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10995846,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,Scott Eckert,RPE Realty,None,None,1,orange,Beresford at 350 S Ocean Blvd Unit 6-A in Boca...
1,PAST SALE,2024-11-06,Condo/Co-op,2150 S Ocean Blvd Unit 2c,Delray Beach,FL,33483.0,1725000.0,3.0,3.5,Bermuda High West One And Two Condo,2352.0,NaN,1979.0,NaN,733.0,1983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Beaches MLS,RX-11005988,N,Y,26.434010,-80.063828,https://www.redfin.com/FL/Delray-Beach/2150-S-...,Nemid Schapira,Signature Ultimate Real Estate LLC,None,None,Judith Troup,Regency Realty Services,None,None,2,blue,Bermuda High West One And Two Condo at 2150 S ...
2,PAST SALE,2024-11-05,Condo/Co-op,1000 Lowry St Ph -C,Delray Beach,FL,33483.0,1450000.0,2.0,2.5,Delray Summit Condo,2110.0,NaN,1966.0,NaN,687.0,2290.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Beaches MLS,RX-11028066,N,Y,26.463100,-80.062729,https://www.redfin.com/FL/Delray-Beach/1000-Lo...,Sally Areson,Dunhill 100 LLC,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,3,blue,Delray Summit Condo at 1000 Lowry St Ph -C in ...
3,PAST SALE,2024-11-06,Condo/Co-op,200 Macfarlane Dr #705,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Seagate Towers,1760.0,NaN,1970.0,NaN,739.0,1522.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,RX-10961305,N,Y,26.457685,-80.063664,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Erin Omiatek,One Sotheby's International Realty,None,None,Mickel Averick,The Corcoran Group,None,None,4,blue,Seagate Towers at 200 Macfarlane Dr #705 in De...
4,PAST SALE,2024-11-08,Condo/Co-op,2000 S Ocean Blvd Unit 9-D,Boca Raton,FL,33432.0,1225000.0,2.0,2.0,Whitehall Condo,1236.0,NaN,1971.0,NaN,991.0,1478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024359,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Stuart Rosenberg,The Keyes Company (PBG),Maude Eaton,The Keyes Company (PBG),Torm Martinson,Campbell&Rosemurgy Real Estate,None,None,5,blue,Whitehall Condo at 2000 S Ocean Blvd Unit 9-D ...
5,PAST SALE,2024-11-05,Condo/Co-op,3210 S Ocean Blvd #604,Highland Beach,FL,33487.0,1065000.0,3.0,3.0,Villa Costa Condo,2108.0,NaN,1991.0,NaN,505.0,2707.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3210-...,Beaches MLS,RX-10982364,N,Y,26.417636,-80.065297,https://www.redfin.com/FL/Highland-Beach/3210-...,Margaret Empsall,John P. O'Grady LLC,Sandra O'Neill-Shine,John P. O'Grady LLC,Amy Blake,John P. O'Grady LLC,Kathya Seckinger,John P. O'Grady LLC,6,blue,Villa Costa Condo at 3210 S Ocean Blvd #604 in...
6,PAST SALE,2024-11-06,Condo/Co-op,550 Okeechobee Blvd #910,West Palm Beach,FL,33401.0,950000.0,2.0,2.0,Cityplace South Tower Condo,1694.0,NaN,2008.0,NaN,561.0,1670.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-11019913,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Thalia Arellano,Sutter & Nugent LLC,None,None,Robert Anderson,"Tortoise Realty Group, LLC",None,None,7,blue,Cityplace South Tower Condo at 550 Okeechobee ...
7,PAST SALE,2024-11-05,Condo/Co-op,1200 S Flagler Dr #1706,West Palm Beach,FL,33401.0,930000.0,2.0,2.0,Trianon Con

## Time on Market Calculator

In [155]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 12) ## List (Earlier) date
date2 = datetime(2024, 11, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

147


## Clean CSV for Datawrapper Chart

In [173]:
chart_df = df_top_ten

In [174]:
chart_df = chart_df.fillna(" ")

In [175]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [176]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [177]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [178]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [179]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [180]:
csv_date_string = today.strftime("%m_%d_%Y")

In [181]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [182]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-05,350 S Ocean Blvd Unit 6-A in Boca Raton,"$3,650,000",3,3.0,"2,523",1977,"$1,447",Beaches MLS,RX-10995846,Adrienne Jankelow Coldwell Banker Realty /Delr...,Scott Eckert RPE Realty
1,2024-11-06,2150 S Ocean Blvd Unit 2c in Delray Beach,"$1,725,000",3,3.5,"2,352",1979,$733,Beaches MLS,RX-11005988,Nemid Schapira Signature Ultimate Real Estate ...,Judith Troup Regency Realty Services
2,2024-11-05,1000 Lowry St Ph -C in Delray Beach,"$1,450,000",2,2.5,"2,110",1966,$687,Beaches MLS,RX-11028066,Sally Areson Dunhill 100 LLC,David Hall Compass Florida LLC Sonia Hall Comp...
3,2024-11-06,200 Macfarlane Dr #705 in Delray Beach,"$1,300,000",2,2.0,"1,760",1970,$739,Beaches MLS,RX-10961305,Erin Omiatek One Sotheby's International Realt...,Mickel Averick The Corcoran Group
4,2024-11-08,2000 S Ocean Blvd Unit 9-D in Boca Raton,"$1,225,000",2,2.0,"1,236",1971,$991,Beaches MLS,RX-11024359,Stuart Rosenberg The Keyes Company (PBG) Maude...,Torm Martinson Campbell&Rosemurgy Real Estate
5,2024-11-05,3210 S Ocean Blvd #604 in Highland Beach,"$1,065,000",3,3.0,"2,108",1991,$505,Beaches MLS,RX-10982364,Margaret Empsall John P. O'Grady LLC Sandra O'...,Amy Blake John P. O'Grady LLC Kathya Seckinger...
6,2024-11-06,550 Okeechobee Blvd #910 in West Palm Beach,"$950,000",2,2.0,"1,694",2008,$561,Beaches MLS,RX-11019913,Thalia Arellano Sutter & Nugent LLC,"Robert Anderson Tortoise Realty Group, LLC"
7,2024-11-05,1200 S Flagler Dr #1706 in West Palm Beach,"$930,000",2,2.0,"1,247",1972,$746,Beaches MLS,F10441726,Chris Cusimano Keller Williams Realty Boca Rat...,Brown Harris Stevens of PB
8,2024-11-06,3908 S Ocean Blvd #564 in Highland Beach,"$800,000",2,2.5,"2,057",1976,$389,MARMLS,A11662556,"Bruce Germinsky BG Realty Services of Florida,...",Angela Camargo Colson AColson Realty Group
9,2024-11-07,2165 Ibis Isle Rd #3 in Palm Beach,"$750,000",2,2.0,"1,485",1969,$505,Beaches MLS,RX-11023956,"Jeffrey Cloninger Sotheby's Intl. Realty, Inc....",Joan Wenzel-Messing Douglas Elliman (Palm Beac...
